<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1


    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65


    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-10 14:13:51
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: -6.70 K (-0.05%)
Current PnL: -13.05 L (-9.09%)
CY Booked + Current PnL: -6.09 L (-4.24%)
-------------------
Total profit:  4.86 L
Total loss:  -17.91 L
-------------------
Total Booked + Current PnL: 20.89 L (17.92%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.08%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.54 L (52.19%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.91%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-221.55,H-SC,98.96,87721.0,-7595.0,7632.0,2.69,-7.97,8.70,0.04,157.0,-1.00,0.64,73.02,OX40N,NTT,MISC
4,APOLLOHOSP,6904.43,8285.00,-18.73,H-LC,89.58,110768.0,7202.0,13503.0,-1.13,6.95,12.19,20.00,32.0,0.53,0.81,22.01,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-21.55,H-LC,72.92,108224.0,3912.0,13625.0,0.36,3.75,12.59,16.81,27.0,0.29,0.79,20.40,X5K,ATH,METALS
71,SBIN,760.30,863.00,-11.51,M-LC,44.79,214902.0,13422.0,13797.0,-0.21,6.66,6.42,13.51,77.0,0.97,1.57,20.10,XY25,NTT,BANKS
87,UJJIVANSFB,52.77,53.00,48.19,M-SC,7.29,128709.0,-13770.0,14390.0,0.27,-9.66,11.18,0.44,247.0,-0.96,0.94,51.91,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,43.75,151610.0,697.0,59750.0,-0.03,0.46,39.41,40.06,103.0,0.01,1.11,24.01,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.91,H-MC,34.38,99529.0,-975.0,31610.0,0.84,-0.97,31.76,30.49,97.0,-0.03,0.73,31.12,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CAMPUS,294.86,393.0,-21.79,M-SC,46.88,162962.0,-4813.0,60654.0,5.80,-2.87,37.22,33.28,206.0,-0.08,1.19,26.17,XY24,NTT,FOOTWEAR
65,RELAXO,902.64,1176.0,-26.70,H-SC,97.40,66285.0,-49253.0,84242.0,3.67,-42.63,127.09,30.28,131.0,-0.58,0.48,29.49,X40N,NTT,FOOTWEAR
69,SAMMAANCAP,170.35,326.0,-169.32,M-SC,84.38,79284.0,-22926.0,116318.0,3.32,-22.43,146.71,91.37,212.0,-0.20,0.58,28.70,XY25,NTT,FINANCE
92,VIPIND,488.80,489.0,-221.55,H-SC,98.96,87721.0,-7595.0,7632.0,2.69,-7.97,8.70,0.04,157.0,-1.00,0.64,73.02,OX40N,NTT,MISC
8,AWL,325.39,485.0,-68.06,H-MC,22.92,221801.0,-46646.0,178328.0,2.32,-17.38,80.40,49.05,98.0,-0.26,1.62,14.04,XY24,NTT,FMCG


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,GREENPANEL,375.16,537.00,210.08,M-SC,78.12,138348.0,-44730.0,123711.0,-2.16,-24.43,89.42,43.14,217.0,-0.36,1.01,30.14,XY24,NTT,MISC
77,STARHEALTH,551.56,761.00,-5.68,M-SC,6.25,154348.0,-45868.0,121889.0,-1.61,-22.91,78.97,37.97,215.0,-0.38,1.13,23.93,XY24,NTT,INSURANCE
12,BANKINDIA,116.91,190.00,-33.74,M-MC,10.42,175409.0,-4399.0,116805.0,-1.60,-2.45,66.59,62.52,173.0,-0.04,1.28,30.81,XR,NTT,BANKS
62,PGHH,13388.00,18062.58,-21.53,X-MC,33.33,198090.0,-2730.0,72858.0,-1.57,-1.36,36.78,34.92,55.0,-0.04,1.45,3.25,X40,ATH,FMCG
28,DIGITIDE,188.38,237.72,NaN,NaN,16.67,48530.0,7275.0,3528.0,-1.56,17.63,7.27,26.19,272.0,2.06,0.35,14.26,XY24,ATH,IT


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-221.55,H-SC,98.96,87721.0,-7595.0,7632.0,2.69,-7.97,8.70,0.04,157.0,-1.00,0.64,73.02,OX40N,NTT,MISC
87,UJJIVANSFB,52.77,53.0,48.19,M-SC,7.29,128709.0,-13770.0,14390.0,0.27,-9.66,11.18,0.44,247.0,-0.96,0.94,51.91,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-28.80,M-SC,31.25,151737.0,-19404.0,19346.0,0.42,-11.34,12.75,-0.03,191.0,-1.00,1.11,9.06,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,90.62,148837.0,-25722.0,25749.0,-0.00,-14.74,17.30,0.02,133.0,-1.00,1.09,29.06,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-18.66,X-MC,5.21,85470.0,-17182.0,17179.0,-0.76,-16.74,20.10,-0.00,59.0,-1.00,0.62,29.92,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-19.02,X-LC,29.17,263375.0,-32559.0,126473.0,-0.21,-11.00,48.02,31.73,1.0,-0.26,1.92,5.42,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.83,X-LC,58.33,292432.0,30831.0,114780.0,-0.31,11.79,39.25,55.67,2.0,0.27,2.13,18.61,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.79,X-LC,62.50,269808.0,-539.0,35156.0,-0.59,-0.20,13.03,12.81,5.0,-0.02,1.97,12.84,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.95,X-LC,77.08,270641.0,41408.0,25873.0,0.03,18.06,9.56,29.35,10.0,1.60,1.98,28.60,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-6.10,X-LC,11.46,79425.0,-30747.0,119821.0,-1.28,-27.91,150.86,80.85,11.0,-0.26,0.58,16.98,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3127.7,NaN,NaN,95.83,78192.0,43743.0,0.0,0.04,126.98,0.00,126.98,274.0,inf,0.57,19.57,AR,ATH,ELECTRICAL
52,JPPOWER,18.73,26.2,-14.39,L-SC,100.00,172210.0,30368.0,26210.0,1.65,21.41,15.22,39.88,262.0,1.16,1.26,74.52,XY24,NTT,POWER
59,MEDANTA,1064.65,1486.0,17.74,X-SC,91.67,251074.0,45597.0,35728.0,1.74,22.19,14.23,39.58,65.0,1.28,1.83,34.54,XY24,NTT,HEALTHCARE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,92.75,M-MC,60.42,231747.0,6785.0,160879.0,0.52,3.02,69.42,74.53,189.0,0.04,1.69,34.55,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.79,29.90,H-SC,66.67,133056.0,7759.0,51360.0,2.06,6.19,38.60,47.18,143.0,0.15,0.97,66.46,MH,ATH,POWER
20,CAMS,3643.00,5250.99,4.31,H-SC,37.50,117236.0,15232.0,29790.0,-0.09,14.93,25.41,44.14,126.0,0.51,0.86,33.60,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2337.55,-4.20,H-SC,93.75,179810.0,15140.0,53943.0,-0.64,9.19,30.00,41.95,124.0,0.28,1.31,17.35,X40N,ATH,AC
95,WIPRO,243.46,420.00,-4.22,M-LC,79.17,164176.0,13231.0,96224.0,-1.12,8.77,58.61,72.51,71.0,0.14,1.20,13.09,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3127.70,NaN,NaN,95.83,78192.0,43743.0,0.0,0.04,126.98,0.00,126.98,274.0,inf,0.57,19.57,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-40.58,M-SC,59.38,195804.0,24183.0,137885.0,-1.18,14.09,70.42,94.43,201.0,0.18,1.43,44.52,XR,NTT,DURABLES
54,KPIGREEN,497.21,731.79,29.90,H-SC,66.67,133056.0,7759.0,51360.0,2.06,6.19,38.60,47.18,143.0,0.15,0.97,66.46,MH,ATH,POWER
95,WIPRO,243.46,420.00,-4.22,M-LC,79.17,164176.0,13231.0,96224.0,-1.12,8.77,58.61,72.51,71.0,0.14,1.20,13.09,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-47.14,H-SC,81.25,137434.0,14098.0,119664.0,0.21,11.43,87.07,108.45,123.0,0.12,1.00,36.34,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.2,NaN,NaN,69.27,18878.0,-30715.0,0.0,0.20,-61.93,0.00,-61.93,270.0,-inf,0.14,26.60,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.0,-55.73,L-SC,18.75,66720.0,-46829.0,86883.0,1.16,-41.24,130.22,35.27,265.0,-0.54,0.49,63.30,XR,NTT,HOTELS
32,ENRIN,1377.95,3127.7,NaN,NaN,95.83,78192.0,43743.0,0.0,0.04,126.98,0.00,126.98,274.0,inf,0.57,19.57,AR,ATH,ELECTRICAL
6,ASIANTILES,75.41,137.0,7211.11,M-SC,65.62,80238.0,-13572.0,90188.0,2.14,-14.47,112.40,81.67,245.0,-0.15,0.59,55.38,XR,NTT,CERAMICS
92,VIPIND,488.80,489.0,-221.55,H-SC,98.96,87721.0,-7595.0,7632.0,2.69,-7.97,8.70,0.04,157.0,-1.00,0.64,73.02,OX40N,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-14.39,L-SC,100.00,172210.0,30368.0,26210.0,1.65,21.41,15.22,39.88,262.0,1.16,1.26,74.52,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-221.55,H-SC,98.96,87721.0,-7595.0,7632.0,2.69,-7.97,8.70,0.04,157.0,-1.00,0.64,73.02,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,6.93,X-SC,97.40,99575.0,-1388.0,95921.0,-0.40,-1.37,96.33,93.63,67.0,-0.01,0.73,48.31,SR,ATH,CHEMICALS
65,RELAXO,902.64,1176.00,-26.70,H-SC,97.40,66285.0,-49253.0,84242.0,3.67,-42.63,127.09,30.28,131.0,-0.58,0.48,29.49,X40N,NTT,FOOTWEAR
32,ENRIN,1377.95,3127.70,NaN,NaN,95.83,78192.0,43743.0,0.0,0.04,126.98,0.00,126.98,274.0,inf,0.57,19.57,AR,ATH,ELECTRICAL


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.99
1,25,41.97
2,50,70.65


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.71
LC    33.72
MC    21.54
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.11
X40      12.89
XY25     12.07
X40N     11.92
XR        9.94
OX40N     8.21
AR        8.17
MH        1.70
X5K       1.41
X200      1.40
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.58
M-SC    15.91
H-LC    14.02
X-LC    12.86
H-MC     9.59
X-MC     7.56
M-LC     5.80
X-SC     4.57
M-MC     4.04
L-SC     2.65
L-LC     1.04
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.24,-0.31,33.85
IT,12.00,-10.41,64.68
FINANCE,8.56,-11.25,51.62
BANKS,7.62,-9.12,53.64
MISC,5.88,-19.52,75.18
PAINTS,5.67,-12.60,34.78
ELECTRICAL,5.45,1.26,45.37
HEALTHCARE,4.75,0.80,28.18
AUTO,4.31,-14.51,64.86


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2629834.0,26
XR,1095610.0,13
AR,1029278.0,10
X40,688553.0,10
X40N,531714.0,11
XY25,466922.0,8
OX40N,306348.0,11
SR,178480.0,2
MH,82970.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1965634.0,20
M-SC,1764551.0,21
H-LC,622471.0,12
X-LC,548064.0,8
H-MC,520267.0,8
X-MC,417103.0,6
M-MC,416936.0,3
X-SC,292593.0,5
M-LC,266153.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      889063.0      6
M-SC       XY24      802803.0      7
H-SC       AR        522339.0      3
M-SC       XR        340729.0      4
M-MC       XY24      300131.0      2
H-MC       XY24      294956.0      3
X-LC       X40       276194.0      3
H-SC       XR        215883.0      3
           X40N      201974.0      4
H-LC       X40N      199629.0      4
M-SC       AR        187900.0      2
           OX40N     165015.0      5
H-LC       X40       159785.0      3
X-MC       X40       157288.0      2
M-LC       XY24      156132.0      2
L-SC       XR        150163.0      2
X-SC       XR        121805.0      1
X-LC       AR        119821.0      1
M-MC       XR        116805.0      1
M-SC       XY25      116318.0      1
X-MC       XY25      105709.0      1
X-LC       XY25      101308.0      3
M-LC       XR         96224.0      1
X-SC       SR         95921.0      1
H-SC       OX40N      85015.0      3
X-MC       XY24       83094.0      1
M-SC       SR         82559.0      1
H-MC       XY25       82355.0      1
H-LC       AR         80443.0      1
           X5K        75232.0      2
M-SC       X40        69227.0      1
H-LC       X200       69193.0      1
H-MC       AR         59750.0      1
L-SC       AR         59025.0      1
L-MC       XR         54001.0      1
X-MC       X40N       53833.0      1
H-SC       MH         51360.0      1
X-LC       X40N       50741.0      1
L-LC       XY25       47435.0      1
X-SC       OX40N      39139.0      2
H-LC       XY24       38189.0      1
X-SC       XY24       35728.0      1
H-MC       MH         31610.0      1
L-SC       XY24       26210.0      1
H-MC       X40        26059.0      1
           X40N       25537.0      1
X-MC       OX40N      17179.0      1
M-LC       XY25       13797.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 40.0 seconds
